# Basic Practices for Gurobi and Cplex

Reference:  
Winston, W. L., & Goldberg, J. B. (2004). Operations research: applications and algorithms (Vol. 3).   
Belmont: Thomson Brooks/Cole.  
(Section 3.6, page 80 - 81)  
Example 10 Project Selection  

<u> 1. Solve this problem with gurobi (Version 1) </u>

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

m = gp.Model('Project Selection V1')

x1 = m.addVar(lb=0, name='x1')
x2 = m.addVar(lb=0, name='x2')
x3 = m.addVar(lb=0, name='x3')
x4 = m.addVar(lb=0, name='x4')
x5 = m.addVar(lb=0, name='x5')

m.addConstr(11*x1 + 53*x2 + 5*x3 + 5*x4 + 29*x5 <= 40)
m.addConstr(3*x1 + 6*x2 + 5*x3 + x4 + 34*x5 <= 20)
m.addConstr(x1 <= 1)
m.addConstr(x2 <= 1)
m.addConstr(x3 <= 1)
m.addConstr(x4 <= 1)
m.addConstr(x5 <= 1)

m.setObjective(13*x1 + 16*x2 + 16*x3 + 14*x4 + 39*x5, GRB.MAXIMIZE)

m.optimize()

print('Objective:', str(m.objVal))

print('x1: ', m.getVarByName('x1').x)
print('x2: ', m.getVarByName('x2').x)
print('x3: ', m.getVarByName('x3').x)
print('x4: ', m.getVarByName('x4').x)
print('x5: ', m.getVarByName('x5').x)
m.dispose()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-19
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 5 columns and 15 nonzeros
Model fingerprint: 0xe4029bee
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 5 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2800000e+02   9.500000e+00   0.000000e+00      0s
       2    5.7449017e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.744901720e+01
Objective: 57.449017199017206
x1:  1.0
x2:  0.20085995085995084
x3:  1.0
x4:  1.0
x5:  0.2880835380835381


<u> 2. Solve this problem with gurobi (Version 2) </u>

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

t0_co = [11, 53, 5, 5, 29]
t1_co = [3, 6, 5, 1, 34]
npv = [13, 16, 16, 14, 39]

m = gp.Model('Project Selection V2')

x = {}
for i in range(5):
    x[i] = m.addVar(obj=npv[i], lb=0, ub=1, name='x'+str(i+1))
    
m.addConstr(sum(t0_co[i]*x[i] for i in range(5)) <= 40)
m.addConstr(sum(t1_co[i]*x[i] for i in range(5)) <= 20)

m.ModelSense = GRB.MAXIMIZE

m.optimize()

print('Objective: ', m.objVal)
    
for v in m.getVars():
    print(v.varName, ': ', v.x)
    
m.dispose()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0xa624fb66
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 4e+01]
Presolve time: 0.01s
Presolved: 2 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2800000e+02   9.500000e+00   0.000000e+00      0s
       3    5.7449017e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.744901720e+01
Objective:  57.449017199017206
x1 :  1.0
x2 :  0.20085995085995087
x3 :  1.0
x4 :  1.0
x5 :  0.2880835380835381


<u> 3. Solve this problem with cplex (Version 3) </u>

In [3]:
import cplex

obj_val = [13, 16, 16, 14, 39]
rhs_val = [40, 20]
lb_val = [0, 0, 0, 0, 0]
ub_val = [1, 1, 1, 1, 1]
sense_val = ['L', 'L']
var_names = ['x1', 'x2', 'x3', 'x4', 'x5']
t0_co = [11, 53, 5, 5, 29]
t1_co = [3, 6, 5, 1, 34]
constr_expr = [[var_names, t0_co], [var_names, t1_co]]

cpx = cplex.Cplex()
cpx.objective.set_sense(cpx.objective.sense.maximize)

cpx.variables.add(obj=obj_val, lb=lb_val, ub=ub_val, names=var_names)

cpx.linear_constraints.add(lin_expr=constr_expr, senses=sense_val, rhs=rhs_val)

cpx.solve()

print('Objective: ', cpx.solution.get_objective_value())

var_opt = cpx.solution.get_values()

for i in range(len(var_names)):
    print(var_names[i], ': ', var_opt[i])  

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            70.181818
Objective:  57.4490171990172
x1 :  1.0
x2 :  0.20085995085995084
x3 :  1.0
x4 :  1.0
x5 :  0.2880835380835381
